In [348]:
'''
Some comments regarding this data:
    1) The data is divided between output labels 0 and 1, and as question says that we need to take last 15 examples as test
       This means only data with output label 0 are there in test data.
    2) In CNN while condensing we need to include a random variable which first randomly adds a data point into the list of 
       condensed data. This gives varying accuracy. We notices that accuracy has three possible values 53.33,60 and 66.667.
    3) If the index =31,38,44,48,49,52,53 is added first to the list of condensed data then the accuracy is highest.
       This highest accuracy is 66.667 which is equivalent to the accuracy given by standard K-NN of scikit learn with k= 5.
'''

'\nSome comments regarding this data:\n    1) The data is divided between output labels 0 and 1, and as question says that we need to take last 15 examples as test\n       This means only data with output label 0 are there in test data.\n    2) In CNN while condensing we need to include a random variable which first randomly adds a data point into the list of \n       condensed data. This gives varying accuracy. We notices that accuracy varies from 53.33 to 66.667\n    3) If the index =31,38,44,48,49,52,53 is added first to the list of condensed data then the accuracy is highest.\n       This highest accuracy is 66.667.\n'

In [349]:
#installing libraries using pip
!pip install numpy
!pip install pandas
!pip install sklearn

In [352]:
#importing libraries
import numpy as np
import pandas as pd 
import math
import random

In [353]:
#Reading the csv file
df = pd.read_csv("sobar-72.csv")
df.head()


behavior_sexualRisk  behavior_eating  behavior_personalHygine  \
0                   10               13                       12   
1                   10               11                       11   
2                   10               15                        3   
3                   10               11                       10   
4                    8               11                        7   

   intention_aggregation  intention_commitment  attitude_consistency  \
0                      4                     7                     9   
1                     10                    14                     7   
2                      2                    14                     8   
3                     10                    15                     7   
4                      8                    10                     7   

   attitude_spontaneity  norm_significantPerson  norm_fulfillment  \
0                    10                       1                 8   
1                     7                       5                 5   
2                    10                       1                 4   
3                     7                       1                 5   
4                     8                       1                 5   

   perception_vulnerability  perception_severity  motivation_strength  \
0                         7                    3                   14   
1                         4                    2                   15   
2                         7                    2                    7   
3                         4                    2                   15   
4                         3                    2                   15   

   motivation_willingness  socialSupport_emotionality  \
0                       8                           5   
1                      13                           7   
2                       3                           3   
3                      13                           7   
4                       5                           3   

   socialSupport_appreciation  socialSupport_instrumental  \
0                           7                          12   
1                           6                           5   
2                           6                          11   
3                           4                           4   
4                           6                          12   

   empowerment_knowledge  empowerment_abilities  empowerment_desires  \
0                     12                     11                    8   
1                      5                      4                    4   
2                      3                      3                   15   
3                      4                      4                    4   
4                      5                      4                    7   

   ca_cervix  
0          1  
1          1  
2          1  
3          1  
4          1

In [354]:
#Separating the input features from the output labels and storing them in numpy arrays to make it easier to use
X = df.iloc[:,0:19]
X_np = np.asarray(X)
Y = df.iloc[:,19:]
Y_np = np.asarray(Y)

In [365]:
#Class CNN which does the work of condensing the data using closest neighbour (1-NN)
class CNN(object):
    
    def CNN(self, X_train, Y_train):
        
        #first we randomly add a data point into our list
        rand = random.randint(0,X_train.shape[0]-1)
        print("rand =", rand)
        list = [(X_train[rand], Y_train[rand])]  #list of the condensed data
        
        #initializing the required variables (making a generator set which helps us iterate through the training data)
        # output variable signifies if there is any data point which doesnt match with its output after condensing the data
        #We keep adding new data points until we reach a point where output != 0(i.e all data points are getting same result with the condensed data)
        output = -1
        gen_set = [i for i in range(Y_train.shape[0])]
        gen_set.remove(rand)
        while(output!=0):
            for i in gen_set:
                output = self.nearest(list,X_train[i], Y_train[i])
                if(output == 1):  #if output= 1 means that this data didnt get the correct output label by the condensed data
                    list.append((X_train[i], Y_train[i])) #as this data point didnt get correct output we add it to the list of condensed data
                    gen_set.remove(i) #we remove the index if it has been added into condensed data list
                    break; #after adding the new data point we again iterate through the data and check if there is any other data point whose output doesnt match
        return list
    
    #this function helps us to find the nearest neighbour
    def nearest(self, list, X_train, Y_train):
        mini , y_final = 10000000, -1
        for x,y in list:
            temp  = np.dot(x-X_train, x-X_train)
            math.sqrt(temp)
            if(temp < mini):
                mini = temp
                y_final = y[0]
        if(Y_train[0] != y_final): #if the output label of nearest neighbour doesnt match then we return 1 or else return 0
            return 1
        return 0

    
#Split the training data 
X_train = X_np[:57] 
Y_train = Y_np[:57]

#making object of the CNN class and calling it
model = CNN()
list = model.CNN(X_np[:57], Y_np[:57])

rand = 31


In [366]:
#class to predict the outputs and print the accuracy
class pred:
    def __init__(self, X_test, Y_test):
        #correct variable stores the number of correctly predicted outputs
        correct = 0
        for i in range(0,X_test.shape[0]):
            y_output = self.pred(list, X_test[i], Y_test[i])
            if(y_output == Y_test[i][0]):
                correct+=1
        print("Accuracy on test set: ",correct*100.00/X_test.shape[0])

    #this function does the knn with k=5 for each data point
    def pred(self, list, X_test, Y_test):
        min ,cnt, majority= 10000000, 0, 0
        distances =[] #distances is a list of tuples of (distance, output_label)
        for x,y in list:
            temp  = np.dot(x-X_test, x-X_test)
            math.sqrt(temp)
            distances.append((temp,y[0]))
        # we sort the distances list on the basis of distance and take majority vote in first 5 elements
        distances = self.Sort_Tuple(distances)
        for i in range(5):
            majority += distances[i][1]
        majority = majority/5.0
        if(majority > 0.5):
            return 1
        else:
            return 0

    def Sort_Tuple(self, tup): 
        tup.sort(key = lambda x: x[0]) 
        return tup 

#Getting the test data
X_test = X_np[57:]
Y_test = Y_np[57:]
correct = pred(X_test, Y_test)


    

Accuracy on test set:  66.66666666666667


In [367]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

Y_train = Y_train.flatten()
clf = KNeighborsClassifier(n_neighbors = 5)
clf.fit(X_train, Y_train)

training_score = clf.score(X_train, Y_train)
Y_test = Y_test.flatten()
test_score = clf.score(X_test, Y_test)
print("Training Score: ", training_score*100)
print("Test Score: ", test_score*100)

Training Score:  85.96491228070175
Test Score:  66.66666666666666
